In [8]:
import numpy as np
import numpy.ma as ma
import math as math
import astropy
from astropy.stats import biweight_location
from astropy.stats import biweight_scale
from astropy.stats import sigma_clip
import os

In [9]:
true_swift_list = ['XTE J1118+480','MAXI J1305-704','MAXI J1348-630','SWIFT J1357.2-0933','GS 1354-64','MAXI J1535-571',
                   'Swift J1539.2-6227','4U 1543-47','MAXI J1543-564','4U 1630-472','XTE J1652-453','GRO J1655-40',
                   'MAXI J1659-152','GX 339-4','GRS 1716-249','IGR J17285-2922','GRS 1739-278','Swift J174510.8-262411',
                   'H 1743-322','SLX 1746-331','IGR J17497-2821','XTE J1752-223','MAXI J1803-298','XTE J1812-182',
                   'XTE J1817-330','XTE J1818-245','SAX J1819.3-2525','MAXI J1820+070','MAXI J1836-194','Swift J1842.5-1124',
                   'EXO 1846-031','XTE J1856+053','XTE J1908+094','Swift J1910.2-0546','GS 2023+338']


true_maxi_list = ['MAXI J0637-430','XTE J1118+480','MAXI J1305-704','MAXI J1348-630','Swift J1357.2-0933','GS 1354-64',
                  'MAXI J1535-571','4U 1543-475','MAXI J1543-564','4U 1630-472','GRO J1655-40','MAXI J1659-152','GX 339-4',
                  'GRS 1716-249','GRS 1739-278','H 1743-322','SLX 1746-331','XTE J1752-223','MAXI J1803-298',
                  'V4641 Sgr','MAXI J1820+070','MAXI J1836-194','Swift J1842.5-1124','EXO 1846-031',
                  'XTE J1856+053','XTE J1908+094','MAXI J1910-057','V404 Cyg']

only_swift_list = ['Swift J1539.2-6227','XTE J1652-453','IGR J17285-2922','Swift J174510.8-262411','IGR J17497-2821',
                   'XTE J1812-182','XTE J1817-330','XTE J1818-245','SAX J1819.3-2525','Swift J1910.2-0546','GS 2023+338']

only_maxi_list= ['MAXI J0637-430','4U 1543-47','V4641 Sgr','MAXI J1910-057','V404 Cyg']

both_list= ['MAXI J1305-704','GS 1354-64','4U 1543-47','4U 1630-472','GRO J1655-40','GRS 1716-249','GRS 1739-278',
            'H 1743-322','SLX 1746-331','MAXI J1803-298','MAXI J1820+070','MAXI J1836-194','Swift J1842.5-1124',
            'EXO 1846-031','XTE J1908+094','XTE J1118+480','MAXI J1348-630','SWIFT J1357.2-0933','MAXI J1535-571',
            'MAXI J1543-564','MAXI J1659-152','GX 339-4','XTE J1752-223','XTE J1856+053']

In [10]:
def sourceName_modifier(sourceName):
    sourceName_ed=sourceName.replace(' ','').replace('-','m').replace('+','p')
    return sourceName_ed

In [11]:
class Solution(object):
    def merge(self, intervals):
        """
        :type intervals: List[Interval]
        :rtype: List[Interval]
        """
        if len(intervals) == 0:
            return []
        self.quicksort(intervals,0,len(intervals)-1)
        #for i in intervals:
            #print(i.start, i.end)
        stack = []
        stack.append(intervals[0])
        for i in range(1,len(intervals)):
            last_element= stack[len(stack)-1]
            if last_element[1] >= intervals[i][0]:
                last_element[1] = max(intervals[i][1],last_element[1])
                stack.pop(len(stack)-1)
                stack.append(last_element)
            else:
                stack.append(intervals[i])
        return stack
    def partition(self,array,start,end):
        pivot_index = start
        for i in range(start,end):
            if array[i][0]<=array[end][0]:
                array[i],array[pivot_index] =array[pivot_index],array[i]
                pivot_index+=1
        array[end],array[pivot_index] =array[pivot_index],array[end]
        return pivot_index
    
    def quicksort(self,array,start,end):
        if start<end:
            partition_index = self.partition(array,start,end)
            self.quicksort(array,start,partition_index-1)
            self.quicksort(array, partition_index + 1, end)
            
ob1 = Solution()

In [12]:
f = open("outbursts_per_sources.txt", "w")
f.write("OUTBURSTS PER SOURCES")
f.write("\n")
f.write("\n")
f.close()
for hohoho in both_list:
    source=sourceName_modifier(hohoho)
    toload1=source+"_SwiftBAT_lc.txt"
    toload2=source+"_MAXI_lc.txt"
    
    
    
    #LOAD SWIFT
    daily_counts1 = np.array(np.loadtxt(toload1, usecols = (1,)))#This will only give the count rate data (column index 1)
    daily_error1 = np.loadtxt(toload1, usecols = (2,))#This will only give the count rate error data (column index 2)
    time1 = np.loadtxt(toload1, usecols = (0,))
    daily_mean1 = biweight_location(daily_counts1)
    daily_std1 = biweight_scale(daily_counts1)
    filtered_daily_data1 = sigma_clip(daily_counts1, sigma = 3)
    daily_detection1 = []
    daily_non_detection1 = []
    daily_detection_time1 = []
    for i in range(len(daily_counts1)):
        if daily_counts1[i] >= daily_mean1 + (3*daily_error1[i]):
            daily_detection1.append(daily_counts1[i])
            daily_detection_time1.append(time1[i])
        elif daily_counts1[i] < daily_mean1 + (3*daily_error1[i]):
            daily_non_detection1.append(daily_counts1[i])
    
    
    #3a DAILY SWIFT
    daily_outbursts1=[]
    if len(daily_detection1)!=0:
        ob=[daily_detection_time1[0]]
        for i in range(len(daily_detection_time1)):
            counter=0
            j=i
            while j<len(daily_detection_time1) and daily_detection_time1[j]<(daily_detection_time1[i]+7): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(daily_detection_time1[i])
                if ob[1]!=ob[0]:
                    daily_outbursts1.append(ob)
                if i<len(daily_detection_time1)-1:
                    ob=[daily_detection_time1[i+1]]
    

    #FOR GETTING THE WEEKLY AVERAGED COUNT DATA  
    data_week_count = []
    data_week_error = []
    u = time1[0]
    i = 0
    wkly_counts1 = []
    wkly_error1 = []
    wkly_times1 = []   
    while i<=len(time1)-1:
        if time1[i]-u<7:
            data_week_count.append(daily_counts1[i])
            data_week_error.append(daily_error1[i])
            i+=1
        else:
            if len(data_week_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
                sum_toAdd = np.sum(1/(np.square(data_week_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            wkly_counts1.append(toAdd)
            wkly_error1.append(toAdd_err)
            wkly_times1.append(u+3)
            u+=7
            data_week_count = []
            data_week_error = []
    if len(data_week_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
        sum_toAdd = np.sum(1/(np.square(data_week_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    wkly_counts1.append(toAdd)
    wkly_error1.append(toAdd_err)
    wkly_times1.append(u+3)   
    weekly_counts1 = ma.masked_invalid(np.array(wkly_counts1))
    weekly_error1 = ma.masked_invalid(np.array(wkly_error1))
    weekly_times1 = np.array(wkly_times1)
    

    #GETTING THE COUNT RATE DATA DIVISION FOR WEEKLY-AVERAGED DATA
    weekly_mean1 = biweight_location(weekly_counts1)
    weekly_std1 = biweight_scale(weekly_counts1)
    filtered_weekly_data1 = sigma_clip(weekly_counts1, sigma = 3)
    weekly_detection1 = []
    weekly_non_detection1 = []
    weekly_detection_time1 = []
    for i in range(len(weekly_counts1)):
        if weekly_counts1[i] >= weekly_mean1 + (3*weekly_error1[i]):
            weekly_detection1.append(weekly_counts1[i])
            weekly_detection_time1.append(weekly_times1[i])
        elif weekly_counts1[i] < weekly_mean1 + (3*weekly_error1[i]):
            weekly_non_detection1.append(weekly_counts1[i])
    
    
    #3a WEEKLY SWIFT
    weekly_outbursts1=[]
    if len(weekly_detection1)!=0:
        ob=[weekly_detection_time1[0]]
        for i in range(len(weekly_detection_time1)):
            counter=0
            j=i
            while j<len(weekly_detection_time1) and weekly_detection_time1[j]<(weekly_detection_time1[i]+21): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(weekly_detection_time1[i])
                if ob[1]!=ob[0]:
                    weekly_outbursts1.append(ob)
                if i<len(weekly_detection_time1)-1:
                    ob=[weekly_detection_time1[i+1]]
                

    #FOR GETTING THE MONTHLY AVERAGED COUNT DATA
    data_month_count = [] #
    data_month_error = []
    u=time1[0]
    i=0
    mth_counts1=[]
    mth_error1 = []
    mth_times1 = []
    while i<=len(time1)-1:
        if time1[i]-u<21:
            data_month_count.append(daily_counts1[i])
            data_month_error.append(daily_error1[i])
            i+=1
        else:
            if len(data_month_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
                sum_toAdd = np.sum(1/(np.square(data_month_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            mth_counts1.append(toAdd)
            mth_error1.append(toAdd_err)
            mth_times1.append(u+10)
            u+=21
            data_month_count = [] 
            data_month_error = []
    if len(data_month_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
        sum_toAdd = np.sum(1/(np.square(data_month_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    mth_counts1.append(toAdd)
    mth_error1.append(toAdd_err)
    mth_times1.append(u+10)
    monthly_counts1= ma.masked_invalid(np.array(mth_counts1))
    monthly_error1 = ma.masked_invalid(np.array(mth_error1))
    monthly_times1 = np.array(mth_times1)
    
    
    #GETTING THE COUNT RATE DATA DIVISION FOR MONTHLY-AVERAGED DATA
    monthly_mean1 = biweight_location(monthly_counts1)
    monthly_std1 = biweight_scale(monthly_counts1)
    filtered_monthly_data1 = sigma_clip(monthly_counts1, sigma = 3)
    monthly_detection1 = []
    monthly_non_detection1 = []
    monthly_detection_time1 = []
    for i in range(len(monthly_counts1)):
        if monthly_counts1[i] >= monthly_mean1 + (3*monthly_error1[i]):
            monthly_detection1.append(monthly_counts1[i])
            monthly_detection_time1.append(monthly_times1[i])
        elif monthly_counts1[i] < monthly_mean1 + (3*monthly_error1[i]):
            monthly_non_detection1.append(monthly_counts1[i])
            
           
    #3a MONTHLY SWIFT
    monthly_outbursts1=[]
    if len(monthly_detection1)!=0:
        ob=[monthly_detection_time1[0]]
        for i in range(len(monthly_detection_time1)):
            counter=0
            j=i
            while j<len(monthly_detection_time1) and monthly_detection_time1[j]<(monthly_detection_time1[i]+63): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(monthly_detection_time1[i])
                if ob[1]!=ob[0]:
                    monthly_outbursts1.append(ob)
                if i<len(monthly_detection_time1)-1:
                    ob=[monthly_detection_time1[i+1]]
                
                

    #LOAD MAXI 24     
    daily_counts2 = np.loadtxt(toload2, usecols = (3,))  #This will only give the count rate data (column index 3)
    daily_error2 = np.loadtxt(toload2, usecols = (4,))  #This will only give the count rate error data (column index 4)
    time2 = np.loadtxt(toload2, usecols = (0,))
    daily_mean2 = biweight_location(daily_counts2)
    daily_std2 = biweight_scale(daily_counts2)
    filtered_daily_data2 = sigma_clip(daily_counts2, sigma = 3)
    daily_detection2 = []
    daily_non_detection2 = []
    daily_detection_time2 = []
    for i in range(len(daily_counts2)):
        if daily_counts2[i] >= daily_mean2 + (3*daily_error2[i]):
            daily_detection2.append(daily_counts2[i])
            daily_detection_time2.append(time2[i])
        elif daily_counts2[i] < daily_mean2 + (3*daily_error2[i]):
            daily_non_detection2.append(daily_counts2[i])

    
    #3a DAILY MAXI
    daily_outbursts2=[]
    if len(daily_detection2)!=0:
        ob=[daily_detection_time2[0]]
        for i in range(len(daily_detection_time2)):
            counter=0
            j=i
            while j<len(daily_detection_time2) and daily_detection_time2[j]<(daily_detection_time2[i]+7): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(daily_detection_time2[i])
                if ob[1]!=ob[0]:
                    daily_outbursts2.append(ob)
                if i<len(daily_detection_time2)-1:
                    ob=[daily_detection_time2[i+1]]
                
                
    #FOR GETTING THE WEEKLY AVERAGED COUNT DATA   
    data_week_count = []
    data_week_error = []
    u = time2[0]
    i = 0
    wkly_counts2 = []
    wkly_error2 = []
    wkly_times2 = []
    while i<=(len(time2)-1):
        if time2[i]-u<7:
            data_week_count.append(daily_counts2[i])
            data_week_error.append(daily_error2[i])
            i+=1
        else:
            if len(data_week_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
                sum_toAdd = np.sum(1/(np.square(data_week_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            wkly_counts2.append(toAdd)
            wkly_error2.append(toAdd_err)
            wkly_times2.append(u+3)
            u+=7
            data_week_count = []
            data_week_error = []
    if len(data_week_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
        sum_toAdd = np.sum(1/(np.square(data_week_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    wkly_counts2.append(toAdd)
    wkly_error2.append(toAdd_err)
    wkly_times2.append(u+3)
    weekly_counts2 = ma.masked_invalid(np.array(wkly_counts2)) #pb is here, 
    weekly_error2 = ma.masked_invalid(np.array(wkly_error2))
    weekly_times2 = np.array(wkly_times2)
    
    #GETTING THE COUNT RATE DATA DIVISION FOR WEEKLY-AVERAGED DATA
    weekly_mean2 = biweight_location(weekly_counts2)
    weekly_std2 = biweight_scale(weekly_counts2)
    filtered_weekly_data2 = sigma_clip(weekly_counts2, sigma = 3)
    weekly_detection2 = []
    weekly_non_detection2 = []
    weekly_detection_time2 = []
    for i in range(len(weekly_counts2)):
        if weekly_counts2[i] >= weekly_mean2 + (3*weekly_error2[i]):
            weekly_detection2.append(weekly_counts2[i])
            weekly_detection_time2.append(weekly_times2[i])
        elif weekly_counts2[i] < weekly_mean2 + (3*weekly_error2[i]):
            weekly_non_detection2.append(weekly_counts2[i])
            
            
    #3a WEEKLY MAXI 24
    weekly_outbursts2=[]
    if len(weekly_detection2)!=0:
        ob=[weekly_detection_time2[0]]
        for i in range(len(weekly_detection_time2)):
            counter=0
            j=i
            while j<len(weekly_detection_time2) and weekly_detection_time2[j]<(weekly_detection_time2[i]+21): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(weekly_detection_time2[i])
                if ob[1]!=ob[0]:
                    weekly_outbursts2.append(ob)
                if i<len(weekly_detection_time2)-1:
                    ob=[weekly_detection_time2[i+1]]
                    
    #FOR GETTING THE MONTHLY AVERAGED COUNT DATA
    data_month_count = [] 
    data_month_error = []
    u=time2[0]
    i=0
    mth_counts2=[]
    mth_error2 = []
    mth_times2 = []       
    while i<=len(time2)-1:
        if time2[i]-u<21:
            data_month_count.append(daily_counts2[i])
            data_month_error.append(daily_error2[i])
            i+=1
        else:
            if len(data_month_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
                sum_toAdd = np.sum(1/(np.square(data_month_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            mth_counts2.append(toAdd)
            mth_error2.append(toAdd_err)
            mth_times2.append(u+10)
            u+=21
            data_month_count = [] 
            data_month_error = []    
    if len(data_month_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
        sum_toAdd = np.sum(1/(np.square(data_month_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    mth_counts2.append(toAdd)
    mth_error2.append(toAdd_err)
    mth_times2.append(u+10)
    monthly_counts2= ma.masked_invalid(np.array(mth_counts2))
    monthly_error2 = ma.masked_invalid(np.array(mth_error2))
    monthly_times2 = np.array(mth_times2)
                    
                    
    #GETTING THE COUNT RATE DATA DIVISION FOR MONTHLY-AVERAGED DATA
    monthly_mean2 = biweight_location(monthly_counts2)
    monthly_std2 = biweight_scale(monthly_counts2)
    filtered_monthly_data2 = sigma_clip(monthly_counts2, sigma = 3)
    monthly_detection2 = []
    monthly_non_detection2 = []
    monthly_detection_time2 = []
    for i in range(len(monthly_counts2)):
        if monthly_counts2[i] >= monthly_mean2 + (3*monthly_error2[i]):
            monthly_detection2.append(monthly_counts2[i])
            monthly_detection_time2.append(monthly_times2[i])
        elif monthly_counts2[i] < monthly_mean2 + (3*monthly_error2[i]):
            monthly_non_detection2.append(monthly_counts2[i])
                    
                    
    #3a MONTHLY MAXI 24 CODE
    monthly_outbursts2=[]
    if len(monthly_detection2)!=0:
        ob=[monthly_detection_time2[0]]
        for i in range(len(monthly_detection_time2)):
            counter=0
            j=i
            while j<len(monthly_detection_time2) and monthly_detection_time2[j]<(monthly_detection_time2[i]+63): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(monthly_detection_time2[i])
                if ob[1]!=ob[0]:
                    monthly_outbursts2.append(ob)
                if i<len(monthly_detection_time2)-1:
                    ob=[monthly_detection_time2[i+1]]
                    
                    
                    
    #LOAD MAXI 410
    daily_counts3 = np.loadtxt(toload2, usecols = (5,))  #This will only give the count rate data (column index 5)
    daily_error3 = np.loadtxt(toload2, usecols = (6,))  #This will only give the count rate error data (column index 6)
    time3 = np.loadtxt(toload2, usecols = (0,))
    daily_mean3 = biweight_location(daily_counts3)
    daily_std3 = biweight_scale(daily_counts3)
    filtered_daily_data3 = sigma_clip(daily_counts3, sigma = 3)
    daily_detection3 = []
    daily_non_detection3 = []
    daily_detection_time3 = []
    for i in range(len(daily_counts3)):
        if daily_counts3[i] >= daily_mean3 + (3*daily_error3[i]):
            daily_detection3.append(daily_counts3[i])
            daily_detection_time3.append(time3[i])
        elif daily_counts3[i] < daily_mean3 + (3*daily_error3[i]):
            daily_non_detection3.append(daily_counts3[i])
                    
    
    #3a DAILY MAXI
    daily_outbursts3=[]
    if len(daily_detection3)!=0:
        ob=[daily_detection_time3[0]]
        for i in range(len(daily_detection_time3)):
            counter=0
            j=i
            while j<len(daily_detection_time3) and daily_detection_time3[j]<(daily_detection_time3[i]+7): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(daily_detection_time3[i])
                if ob[1]!=ob[0]:
                    daily_outbursts3.append(ob)
                if i<len(daily_detection_time3)-1:
                    ob=[daily_detection_time3[i+1]]

              
    #FOR GETTING THE WEEKLY AVERAGED COUNT DATA                      
    data_week_count = []
    data_week_error = []
    u = time3[0]
    i = 0
    wkly_counts3 = []
    wkly_error3 = []
    wkly_times3 = []  
    while i<=(len(time3)-1):
        if time3[i]-u<7:
            data_week_count.append(daily_counts3[i])
            data_week_error.append(daily_error3[i])
            i+=1
        else:
            if len(data_week_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
                sum_toAdd = np.sum(1/(np.square(data_week_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            wkly_counts3.append(toAdd)
            wkly_error3.append(toAdd_err)
            wkly_times3.append(u+3)
            u+=7
            data_week_count = []
            data_week_error = []    
    if len(data_week_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
        sum_toAdd = np.sum(1/(np.square(data_week_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    wkly_counts3.append(toAdd)
    wkly_error3.append(toAdd_err)
    wkly_times3.append(u+3)    
    weekly_counts3 = ma.masked_invalid(np.array(wkly_counts3))
    weekly_error3 = ma.masked_invalid(np.array(wkly_error3))
    weekly_times3 = np.array(wkly_times3)
                    
                    
    #GETTING THE COUNT RATE DATA DIVISION FOR WEEKLY-AVERAGED DATA
    weekly_mean3 = biweight_location(weekly_counts3)
    weekly_std3 = biweight_scale(weekly_counts3)
    filtered_weekly_data3 = sigma_clip(weekly_counts3, sigma = 3)
    weekly_detection3 = []
    weekly_non_detection3 = []
    weekly_detection_time3 = []
    for i in range(len(weekly_counts3)):
        if weekly_counts3[i] >= weekly_mean3 + (3*weekly_error3[i]):
            weekly_detection3.append(weekly_counts3[i])
            weekly_detection_time3.append(weekly_times3[i])
        elif weekly_counts3[i] < weekly_mean3 + (3*weekly_error3[i]):
            weekly_non_detection3.append(weekly_counts3[i])
                    
                    
    #3a WEEKLY MAXI 410 
    weekly_outbursts3=[]
    if len(weekly_detection3)!=0:
        ob=[weekly_detection_time3[0]]
        for i in range(len(weekly_detection_time3)):
            counter=0
            j=i
            while j<len(weekly_detection_time3) and weekly_detection_time3[j]<(weekly_detection_time3[i]+21): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(weekly_detection_time3[i])
                if ob[1]!=ob[0]:
                    weekly_outbursts3.append(ob)
                if i<len(weekly_detection_time3)-1:
                    ob=[weekly_detection_time3[i+1]]
                    

    #FOR GETTING THE MONTHLY AVERAGED COUNT DATA                   
    data_month_count = [] 
    data_month_error = []
    u=time3[0]
    i=0
    mth_counts3=[]
    mth_error3 = []
    mth_times3 = []       
    while i<=len(time3)-1:
        if time3[i]-u<21:
            data_month_count.append(daily_counts3[i])
            data_month_error.append(daily_error3[i])
            i+=1
        else:
            if len(data_month_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
                sum_toAdd = np.sum(1/(np.square(data_month_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            mth_counts3.append(toAdd)
            mth_error3.append(toAdd_err)
            mth_times3.append(u+10)
            u+=21
            data_month_count = []
            data_month_error = []
    if len(data_month_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
        sum_toAdd = np.sum(1/(np.square(data_month_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    mth_counts3.append(toAdd)
    mth_error3.append(toAdd_err)
    mth_times3.append(u+10)
    monthly_counts3= ma.masked_invalid(np.array(mth_counts3))
    monthly_error3 = ma.masked_invalid(np.array(mth_error3))
    monthly_times3 = np.array(mth_times3)
                    
                    
    #GETTING THE COUNT RATE DATA DIVISION FOR MONTHLY-AVERAGED DATA
    monthly_mean3 = biweight_location(monthly_counts3)
    monthly_std3 = biweight_scale(monthly_counts3)
    filtered_monthly_data3 = sigma_clip(monthly_counts3, sigma = 3)
    monthly_detection3 = []
    monthly_non_detection3 = []
    monthly_detection_time3 = []
    for i in range(len(monthly_counts3)):
        if monthly_counts3[i] >= monthly_mean3 + (3*monthly_error3[i]):
            monthly_detection3.append(monthly_counts3[i])
            monthly_detection_time3.append(monthly_times3[i])
        elif monthly_counts3[i] < monthly_mean3 + (3*monthly_error3[i]):
            monthly_non_detection3.append(monthly_counts3[i])
                    
                    
    #3a MONTHLY MAXI 410
    monthly_outbursts3=[]
    if len(monthly_detection3)!=0:
        ob=[monthly_detection_time3[0]]
        for i in range(len(monthly_detection_time3)):
            counter=0
            j=i
            while j<len(monthly_detection_time3) and monthly_detection_time3[j]<(monthly_detection_time3[i]+63): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(monthly_detection_time3[i])
                if ob[1]!=ob[0]:
                    monthly_outbursts3.append(ob)
                if i<len(monthly_detection_time3)-1:
                    ob=[monthly_detection_time3[i+1]]
                    
                    
                    
    #LOAD MAXI 210
    daily_counts4 = daily_counts3+daily_counts2
    daily_error4 = daily_error3+daily_error2
    time4 = time3
    daily_mean4 = biweight_location(daily_counts4)
    daily_std4 = biweight_scale(daily_counts4)
    filtered_daily_data4 = sigma_clip(daily_counts4, sigma = 3)
    daily_detection4 = []
    daily_non_detection4 = []
    daily_detection_time4 = []
    for i in range(len(daily_counts4)):
        if daily_counts4[i] >= daily_mean4 + (3*daily_error4[i]):
            daily_detection4.append(daily_counts4[i])
            daily_detection_time4.append(time4[i])
        elif daily_counts4[i] < daily_mean4 + (3*daily_error4[i]):
            daily_non_detection4.append(daily_counts4[i])
                    
                    
    #3a DAILY MAXI
    daily_outbursts4=[]
    if len(daily_detection4)!=0:
        ob=[daily_detection_time4[0]]
        for i in range(len(daily_detection_time4)):
            counter=0
            j=i
            while j<len(daily_detection_time4) and daily_detection_time4[j]<(daily_detection_time4[i]+7): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(daily_detection_time4[i])
                if ob[1]!=ob[0]:
                    daily_outbursts4.append(ob)
                if i<len(daily_detection_time4)-1:
                    ob=[daily_detection_time4[i+1]]
                    
                    
    #FOR GETTING THE WEEKLY AVERAGED COUNT DATA   
    data_week_count = []
    data_week_error = []
    u = time4[0]
    i = 0
    wkly_counts4 = []
    wkly_error4 = []
    wkly_times4 = []    
    while i<=(len(time4)-1):
        if time4[i]-u<7:
            data_week_count.append(daily_counts4[i])
            data_week_error.append(daily_error4[i])
            i+=1
        else:
            if len(data_week_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
                sum_toAdd = np.sum(1/(np.square(data_week_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            wkly_counts4.append(toAdd)
            wkly_error4.append(toAdd_err)
            wkly_times4.append(u+3)
            u+=7
            data_week_count = []
            data_week_error = []
    if len(data_week_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
        sum_toAdd = np.sum(1/(np.square(data_week_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    wkly_counts4.append(toAdd)
    wkly_error4.append(toAdd_err)
    wkly_times4.append(u+3)
    weekly_counts4 = ma.masked_invalid(np.array(wkly_counts4))
    weekly_error4 = ma.masked_invalid(np.array(wkly_error4))
    weekly_times4 = np.array(wkly_times4)
                    
                    
    #GETTING THE COUNT RATE DATA DIVISION FOR WEEKLY-AVERAGED DATA
    weekly_mean4 = biweight_location(weekly_counts4)
    weekly_std4 = biweight_scale(weekly_counts4)
    filtered_weekly_data4 = sigma_clip(weekly_counts4, sigma = 3)
    weekly_detection4 = []
    weekly_non_detection4 = []
    weekly_detection_time4 = []
    for i in range(len(weekly_counts4)):
        if weekly_counts4[i] >= weekly_mean4 + (3*weekly_error4[i]):
            weekly_detection4.append(weekly_counts4[i])
            weekly_detection_time4.append(weekly_times4[i])
        elif weekly_counts4[i] < weekly_mean4 + (3*weekly_error4[i]):
            weekly_non_detection4.append(weekly_counts4[i])
                    
                    
    #3a WEEKLY MAXI 210
    weekly_outbursts4=[]
    if len(weekly_detection4)!=0:
        ob=[weekly_detection_time4[0]]
        for i in range(len(weekly_detection_time4)):
            counter=0
            j=i
            while j<len(weekly_detection_time4) and weekly_detection_time4[j]<(weekly_detection_time4[i]+21): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(weekly_detection_time4[i])
                if ob[1]!=ob[0]:
                    weekly_outbursts4.append(ob)
                if i<len(weekly_detection_time4)-1:
                    ob=[weekly_detection_time4[i+1]]
                    
                    
    #FOR GETTING THE MONTHLY AVERAGED COUNT DATA   
    data_month_count = []
    data_month_error = []
    u=time4[0]
    i=0
    mth_counts4=[]
    mth_error4 = []
    mth_times4 = []     
    while i<=len(time4)-1:
        if time4[i]-u<21:
            data_month_count.append(daily_counts4[i])
            data_month_error.append(daily_error4[i])
            i+=1
        else:
            if len(data_month_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
                sum_toAdd = np.sum(1/(np.square(data_month_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            mth_counts4.append(toAdd)
            mth_error4.append(toAdd_err)
            mth_times4.append(u+10)
            u+=21
            data_month_count = []
            data_month_error = []
    if len(data_month_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
        sum_toAdd = np.sum(1/(np.square(data_month_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    mth_counts4.append(toAdd)
    mth_error4.append(toAdd_err)
    mth_times4.append(u+10)
    monthly_counts4= ma.masked_invalid(np.array(mth_counts4))
    monthly_error4 = ma.masked_invalid(np.array(mth_error4))
    monthly_times4 = np.array(mth_times4)
                    
                    
    #GETTING THE COUNT RATE DATA DIVISION FOR MONTHLY-AVERAGED DATA
    monthly_mean4 = biweight_location(monthly_counts4)
    monthly_std4 = biweight_scale(monthly_counts4)
    filtered_monthly_data4 = sigma_clip(monthly_counts4, sigma = 3)
    monthly_detection4 = []
    monthly_non_detection4 = []
    monthly_detection_time4 = []
    for i in range(len(monthly_counts4)):
        if monthly_counts4[i] >= monthly_mean4 + (3*monthly_error4[i]):
            monthly_detection4.append(monthly_counts4[i])
            monthly_detection_time4.append(monthly_times4[i])
        elif monthly_counts4[i] < monthly_mean4 + (3*monthly_error4[i]):
            monthly_non_detection4.append(monthly_counts4[i])
                    
                    
    #3a MONTHLY MAXI 210
    monthly_outbursts4=[]
    if len(monthly_detection4)!=0:
        ob=[monthly_detection_time4[0]]
        for i in range(len(monthly_detection_time4)):
            counter=0
            j=i
            while j<len(monthly_detection_time4) and monthly_detection_time4[j]<(monthly_detection_time4[i]+63): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(monthly_detection_time4[i])
                if ob[1]!=ob[0]:
                    monthly_outbursts4.append(ob)
                if i<len(monthly_detection_time4)-1:
                    ob=[monthly_detection_time4[i+1]]
                    
                    
    #FINAL SWIFT OUTBURST LIST
    swift_combo = daily_outbursts1 + weekly_outbursts1 + monthly_outbursts1
    final_Swift = ob1.merge(swift_combo)

    #TOTAL MAXI DAILY OUTBURST LIST
    MAXI_daily_combo = daily_outbursts2 + daily_outbursts3 + daily_outbursts4
    total_MAXI_daily = ob1.merge(MAXI_daily_combo)

    #TOTAL MAXI WEEKLY OUTBURST LIST
    MAXI_weekly_combo = weekly_outbursts2 + weekly_outbursts3 + weekly_outbursts4
    total_MAXI_weekly = ob1.merge(MAXI_weekly_combo)

    #TOTAL MAXI MONTHLY OUTBURST LIST
    MAXI_monthly_combo = monthly_outbursts2 + monthly_outbursts3 + monthly_outbursts4
    total_MAXI_monthly = ob1.merge(MAXI_monthly_combo) 

    #TOTAL MAXI 2-4keV OUTBURST LIST
    MAXI_24_combo = daily_outbursts2 + weekly_outbursts2 + monthly_outbursts2
    total_MAXI_24 = ob1.merge(MAXI_24_combo)    

    #TOTAL MAXI 4-10keV OUTBURST LIST
    MAXI_410_combo = daily_outbursts3 + weekly_outbursts3 + monthly_outbursts3
    total_MAXI_410 = ob1.merge(MAXI_410_combo)    

    #TOTAL MAXI 2-10keV OUTBURST LIST
    MAXI_210_combo = daily_outbursts4 + weekly_outbursts4 + monthly_outbursts4
    total_MAXI_210 = ob1.merge(MAXI_210_combo) 

    #TOTAL DAILY OUTBURST LIST
    daily_combo = daily_outbursts1 + total_MAXI_daily
    final_daily = ob1.merge(daily_combo)
                    
    #TOTAL WEEKLY OUTBURST LIST
    weekly_combo = weekly_outbursts1 + total_MAXI_weekly
    final_weekly = ob1.merge(weekly_combo)

    #TOTAL MONTHLY OUTBURST LIST
    monthly_combo = monthly_outbursts1 + total_MAXI_monthly
    final_monthly = ob1.merge(monthly_combo)

    #FINAL MAXI OUTBURST LIST
    energy_combo = total_MAXI_24 + total_MAXI_410 + total_MAXI_210
    final_MAXI = ob1.merge(energy_combo)    

    #TRUE FINAL OUTBURST LIST OF BOTH SWIFT AND MAXI
    data_combo = final_Swift + final_MAXI
    final_outbursts = ob1.merge(data_combo)
                          
    with open('outbursts_per_sources.txt', 'a') as f:
        f.write(str(hohoho))
        f.write(":")
        f.write('\n')
        f.write("Outbursts from all MAXI daily data:")
        f.write(str(total_MAXI_daily))
        f.write('\n')
        f.write("Outbursts from all MAXI weekly data:")
        f.write(str(total_MAXI_weekly))
        f.write('\n')
        f.write("Outbursts from all MAXI monthly data:")
        f.write(str(total_MAXI_monthly))
        f.write('\n')
        f.write("Outbursts from Swift/BAT daily data:")
        f.write(str(daily_outbursts1))
        f.write('\n')
        f.write("Outbursts from Swift/BAT weekly data:")
        f.write(str(weekly_outbursts1))
        f.write('\n')
        f.write("Outbursts from Swift/BAT monthly data:")
        f.write(str(monthly_outbursts1))
        f.write('\n')
        f.write("Outbursts from all daily data:")
        f.write(str(final_daily))
        f.write('\n')
        f.write("Outbursts from all weekly data:")
        f.write(str(final_weekly))
        f.write('\n')
        f.write("Outbursts from all monthly data:")
        f.write(str(final_monthly))
        f.write('\n')
        f.write("Outbursts from Swift/BAT 15 - 50 KeV data:")
        f.write(str(final_Swift))
        f.write('\n')
        f.write("Outbursts from MAXI 2 - 4 KeV data:")
        f.write(str(total_MAXI_24))
        f.write('\n')
        f.write("Outbursts from MAXI 4 - 10 KeV data:")
        f.write(str(total_MAXI_410))
        f.write('\n')
        f.write("Outbursts from MAXI 2 - 10 KeV data:")
        f.write(str(total_MAXI_210))
        f.write('\n')
        f.write("Outbursts from all MAXI data:")
        f.write(str(final_MAXI))
        f.write('\n')
        f.write("Outbursts from all data:")
        f.write(str(final_outbursts))
        f.write('\n')
        f.write('\n')
        f.write('\n')
        f.write('\n')

C:\Users\ngutm\AppData\Local\Temp\ipykernel_20012\3921575511.py:249: RuntimeWarning: divide by zero encountered in true_divide
  toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
C:\Users\ngutm\anaconda3\lib\site-packages\numpy\lib\function_base.py:412: RuntimeWarning: invalid value encountered in multiply
  avg = np.multiply(a, wgt, dtype=result_dtype).sum(axis)/scl
C:\Users\ngutm\AppData\Local\Temp\ipykernel_20012\3921575511.py:250: RuntimeWarning: divide by zero encountered in true_divide
  sum_toAdd = np.sum(1/(np.square(data_week_error)))
C:\Users\ngutm\AppData\Local\Temp\ipykernel_20012\3921575511.py:320: RuntimeWarning: divide by zero encountered in true_divide
  toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
C:\Users\ngutm\anaconda3\lib\site-packages\numpy\lib\function_base.py:412: RuntimeWarning: invalid value encountered in multiply
  avg = np.multiply(a, wgt, dtype=result_dtype).sum(axis)/scl
C:\Users\ngutm\AppData\Loc

In [13]:
f = open("outbursts_per_sources.txt", "a")
f.write("The following only have data from Swift/BAT")
f.write(":")
f.write("\n")
f.write("\n")
f.close()

for hohoho in only_swift_list:
    source=sourceName_modifier(hohoho)
    toload1=source+"_SwiftBAT_lc.txt"
    
    
    
    #LOAD SWIFT
    daily_counts1 = np.array(np.loadtxt(toload1, usecols = (1,)))#This will only give the count rate data (column index 1)
    daily_error1 = np.loadtxt(toload1, usecols = (2,))#This will only give the count rate error data (column index 2)
    time1 = np.loadtxt(toload1, usecols = (0,))
    daily_mean1 = biweight_location(daily_counts1)
    daily_std1 = biweight_scale(daily_counts1)
    filtered_daily_data1 = sigma_clip(daily_counts1, sigma = 3)
    daily_detection1 = []
    daily_non_detection1 = []
    daily_detection_time1 = []
    for i in range(len(daily_counts1)):
        if daily_counts1[i] >= daily_mean1 + (3*daily_error1[i]):
            daily_detection1.append(daily_counts1[i])
            daily_detection_time1.append(time1[i])
        elif daily_counts1[i] < daily_mean1 + (3*daily_error1[i]):
            daily_non_detection1.append(daily_counts1[i])
    
    
    #3a DAILY SWIFT
    daily_outbursts1=[]
    if len(daily_detection1)!=0:
        ob=[daily_detection_time1[0]]
        for i in range(len(daily_detection_time1)):
            counter=0
            j=i
            while j<len(daily_detection_time1) and daily_detection_time1[j]<(daily_detection_time1[i]+7): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(daily_detection_time1[i])
                if ob[1]!=ob[0]:
                    daily_outbursts1.append(ob)
                if i<len(daily_detection_time1)-1:
                    ob=[daily_detection_time1[i+1]]
    

    #FOR GETTING THE WEEKLY AVERAGED COUNT DATA  
    data_week_count = []
    data_week_error = []
    u = time1[0]
    i = 0
    wkly_counts1 = []
    wkly_error1 = []
    wkly_times1 = []   
    while i<=len(time1)-1:
        if time1[i]-u<7:
            data_week_count.append(daily_counts1[i])
            data_week_error.append(daily_error1[i])
            i+=1
        else:
            if len(data_week_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
                sum_toAdd = np.sum(1/(np.square(data_week_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            wkly_counts1.append(toAdd)
            wkly_error1.append(toAdd_err)
            wkly_times1.append(u+3)
            u+=7
            data_week_count = []
            data_week_error = []
    if len(data_week_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
        sum_toAdd = np.sum(1/(np.square(data_week_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    wkly_counts1.append(toAdd)
    wkly_error1.append(toAdd_err)
    wkly_times1.append(u+3)   
    weekly_counts1 = ma.masked_invalid(np.array(wkly_counts1))
    weekly_error1 = ma.masked_invalid(np.array(wkly_error1))
    weekly_times1 = np.array(wkly_times1)
    

    #GETTING THE COUNT RATE DATA DIVISION FOR WEEKLY-AVERAGED DATA
    weekly_mean1 = biweight_location(weekly_counts1)
    weekly_std1 = biweight_scale(weekly_counts1)
    filtered_weekly_data1 = sigma_clip(weekly_counts1, sigma = 3)
    weekly_detection1 = []
    weekly_non_detection1 = []
    weekly_detection_time1 = []
    for i in range(len(weekly_counts1)):
        if weekly_counts1[i] >= weekly_mean1 + (3*weekly_error1[i]):
            weekly_detection1.append(weekly_counts1[i])
            weekly_detection_time1.append(weekly_times1[i])
        elif weekly_counts1[i] < weekly_mean1 + (3*weekly_error1[i]):
            weekly_non_detection1.append(weekly_counts1[i])
    
    
    #3a WEEKLY SWIFT
    weekly_outbursts1=[]
    if len(weekly_detection1)!=0:
        ob=[weekly_detection_time1[0]]
        for i in range(len(weekly_detection_time1)):
            counter=0
            j=i
            while j<len(weekly_detection_time1) and weekly_detection_time1[j]<(weekly_detection_time1[i]+21): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(weekly_detection_time1[i])
                if ob[1]!=ob[0]:
                    weekly_outbursts1.append(ob)
                if i<len(weekly_detection_time1)-1:
                    ob=[weekly_detection_time1[i+1]]
                

    #FOR GETTING THE MONTHLY AVERAGED COUNT DATA
    data_month_count = [] #
    data_month_error = []
    u=time1[0]
    i=0
    mth_counts1=[]
    mth_error1 = []
    mth_times1 = []
    while i<=len(time1)-1:
        if time1[i]-u<21:
            data_month_count.append(daily_counts1[i])
            data_month_error.append(daily_error1[i])
            i+=1
        else:
            if len(data_month_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
                sum_toAdd = np.sum(1/(np.square(data_month_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            mth_counts1.append(toAdd)
            mth_error1.append(toAdd_err)
            mth_times1.append(u+10)
            u+=21
            data_month_count = [] 
            data_month_error = []
    if len(data_month_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
        sum_toAdd = np.sum(1/(np.square(data_month_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    mth_counts1.append(toAdd)
    mth_error1.append(toAdd_err)
    mth_times1.append(u+10)
    monthly_counts1= ma.masked_invalid(np.array(mth_counts1))
    monthly_error1 = ma.masked_invalid(np.array(mth_error1))
    monthly_times1 = np.array(mth_times1)
    
    
    #GETTING THE COUNT RATE DATA DIVISION FOR MONTHLY-AVERAGED DATA
    monthly_mean1 = biweight_location(monthly_counts1)
    monthly_std1 = biweight_scale(monthly_counts1)
    filtered_monthly_data1 = sigma_clip(monthly_counts1, sigma = 3)
    monthly_detection1 = []
    monthly_non_detection1 = []
    monthly_detection_time1 = []
    for i in range(len(monthly_counts1)):
        if monthly_counts1[i] >= monthly_mean1 + (3*monthly_error1[i]):
            monthly_detection1.append(monthly_counts1[i])
            monthly_detection_time1.append(monthly_times1[i])
        elif monthly_counts1[i] < monthly_mean1 + (3*monthly_error1[i]):
            monthly_non_detection1.append(monthly_counts1[i])
            
           
    #3a MONTHLY SWIFT
    monthly_outbursts1=[]
    if len(monthly_detection1)!=0:
        ob=[monthly_detection_time1[0]]
        for i in range(len(monthly_detection_time1)):
            counter=0
            j=i
            while j<len(monthly_detection_time1) and monthly_detection_time1[j]<(monthly_detection_time1[i]+63): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(monthly_detection_time1[i])
                if ob[1]!=ob[0]:
                    monthly_outbursts1.append(ob)
                if i<len(monthly_detection_time1)-1:
                    ob=[monthly_detection_time1[i+1]]
                    
                    
    #FINAL SWIFT OUTBURST LIST
    swift_combo = daily_outbursts1 + weekly_outbursts1 + monthly_outbursts1
    final_Swift = ob1.merge(swift_combo) 

    #TOTAL DAILY OUTBURST LIST
    final_daily = daily_outbursts1
                    
    #TOTAL WEEKLY OUTBURST LIST
    final_weekly = ob1.merge(weekly_outbursts1)

    #TOTAL MONTHLY OUTBURST LIST
    final_monthly = ob1.merge(monthly_outbursts1)
                          
    with open('outbursts_per_sources.txt', 'a') as f:
        f.write(str(hohoho))
        f.write(":")
        f.write('\n')
        f.write("Outbursts from Swift/BAT daily data:")
        f.write(str(daily_outbursts1))
        f.write('\n')
        f.write("Outbursts from Swift/BAT weekly data:")
        f.write(str(weekly_outbursts1))
        f.write('\n')
        f.write("Outbursts from Swift/BAT monthly data:")
        f.write(str(monthly_outbursts1))
        f.write('\n')
        f.write("Outbursts from all data:")
        f.write(str(final_Swift))
        f.write('\n')
        f.write('\n')
        f.write('\n')
        f.write('\n')

In [14]:
f = open("outbursts_per_sources.txt", "a")
f.write("The following only have data from MAXI")
f.write(":")
f.write("\n")
f.write("\n")
f.close()

for hohoho in only_maxi_list:
    source=sourceName_modifier(hohoho)
    toload2=source+"_MAXI_lc.txt"                
                

    #LOAD MAXI 24     
    daily_counts2 = np.loadtxt(toload2, usecols = (3,))  #This will only give the count rate data (column index 3)
    daily_error2 = np.loadtxt(toload2, usecols = (4,))  #This will only give the count rate error data (column index 4)
    time2 = np.loadtxt(toload2, usecols = (0,))
    daily_mean2 = biweight_location(daily_counts2)
    daily_std2 = biweight_scale(daily_counts2)
    filtered_daily_data2 = sigma_clip(daily_counts2, sigma = 3)
    daily_detection2 = []
    daily_non_detection2 = []
    daily_detection_time2 = []
    for i in range(len(daily_counts2)):
        if daily_counts2[i] >= daily_mean2 + (3*daily_error2[i]):
            daily_detection2.append(daily_counts2[i])
            daily_detection_time2.append(time2[i])
        elif daily_counts2[i] < daily_mean2 + (3*daily_error2[i]):
            daily_non_detection2.append(daily_counts2[i])

    
    #3a DAILY MAXI
    daily_outbursts2=[]
    if len(daily_detection2)!=0:
        ob=[daily_detection_time2[0]]
        for i in range(len(daily_detection_time2)):
            counter=0
            j=i
            while j<len(daily_detection_time2) and daily_detection_time2[j]<(daily_detection_time2[i]+7): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(daily_detection_time2[i])
                if ob[1]!=ob[0]:
                    daily_outbursts2.append(ob)
                if i<len(daily_detection_time2)-1:
                    ob=[daily_detection_time2[i+1]]
                
                
    #FOR GETTING THE WEEKLY AVERAGED COUNT DATA   
    data_week_count = []
    data_week_error = []
    u = time2[0]
    i = 0
    wkly_counts2 = []
    wkly_error2 = []
    wkly_times2 = []
    while i<=(len(time2)-1):
        if time2[i]-u<7:
            data_week_count.append(daily_counts2[i])
            data_week_error.append(daily_error2[i])
            i+=1
        else:
            if len(data_week_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
                sum_toAdd = np.sum(1/(np.square(data_week_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            wkly_counts2.append(toAdd)
            wkly_error2.append(toAdd_err)
            wkly_times2.append(u+3)
            u+=7
            data_week_count = []
            data_week_error = []
    if len(data_week_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
        sum_toAdd = np.sum(1/(np.square(data_week_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    wkly_counts2.append(toAdd)
    wkly_error2.append(toAdd_err)
    wkly_times2.append(u+3)
    weekly_counts2 = ma.masked_invalid(np.array(wkly_counts2)) #pb is here, 
    weekly_error2 = ma.masked_invalid(np.array(wkly_error2))
    weekly_times2 = np.array(wkly_times2)
    
    #GETTING THE COUNT RATE DATA DIVISION FOR WEEKLY-AVERAGED DATA
    weekly_mean2 = biweight_location(weekly_counts2)
    weekly_std2 = biweight_scale(weekly_counts2)
    filtered_weekly_data2 = sigma_clip(weekly_counts2, sigma = 3)
    weekly_detection2 = []
    weekly_non_detection2 = []
    weekly_detection_time2 = []
    for i in range(len(weekly_counts2)):
        if weekly_counts2[i] >= weekly_mean2 + (3*weekly_error2[i]):
            weekly_detection2.append(weekly_counts2[i])
            weekly_detection_time2.append(weekly_times2[i])
        elif weekly_counts2[i] < weekly_mean2 + (3*weekly_error2[i]):
            weekly_non_detection2.append(weekly_counts2[i])
            
            
    #3a WEEKLY MAXI 24
    weekly_outbursts2=[]
    if len(weekly_detection2)!=0:
        ob=[weekly_detection_time2[0]]
        for i in range(len(weekly_detection_time2)):
            counter=0
            j=i
            while j<len(weekly_detection_time2) and weekly_detection_time2[j]<(weekly_detection_time2[i]+21): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(weekly_detection_time2[i])
                if ob[1]!=ob[0]:
                    weekly_outbursts2.append(ob)
                if i<len(weekly_detection_time2)-1:
                    ob=[weekly_detection_time2[i+1]]
                    
    #FOR GETTING THE MONTHLY AVERAGED COUNT DATA
    data_month_count = [] 
    data_month_error = []
    u=time2[0]
    i=0
    mth_counts2=[]
    mth_error2 = []
    mth_times2 = []       
    while i<=len(time2)-1:
        if time2[i]-u<21:
            data_month_count.append(daily_counts2[i])
            data_month_error.append(daily_error2[i])
            i+=1
        else:
            if len(data_month_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
                sum_toAdd = np.sum(1/(np.square(data_month_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            mth_counts2.append(toAdd)
            mth_error2.append(toAdd_err)
            mth_times2.append(u+10)
            u+=21
            data_month_count = [] 
            data_month_error = []    
    if len(data_month_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
        sum_toAdd = np.sum(1/(np.square(data_month_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    mth_counts2.append(toAdd)
    mth_error2.append(toAdd_err)
    mth_times2.append(u+10)
    monthly_counts2= ma.masked_invalid(np.array(mth_counts2))
    monthly_error2 = ma.masked_invalid(np.array(mth_error2))
    monthly_times2 = np.array(mth_times2)
                    
                    
    #GETTING THE COUNT RATE DATA DIVISION FOR MONTHLY-AVERAGED DATA
    monthly_mean2 = biweight_location(monthly_counts2)
    monthly_std2 = biweight_scale(monthly_counts2)
    filtered_monthly_data2 = sigma_clip(monthly_counts2, sigma = 3)
    monthly_detection2 = []
    monthly_non_detection2 = []
    monthly_detection_time2 = []
    for i in range(len(monthly_counts2)):
        if monthly_counts2[i] >= monthly_mean2 + (3*monthly_error2[i]):
            monthly_detection2.append(monthly_counts2[i])
            monthly_detection_time2.append(monthly_times2[i])
        elif monthly_counts2[i] < monthly_mean2 + (3*monthly_error2[i]):
            monthly_non_detection2.append(monthly_counts2[i])
                    
                    
    #3a MONTHLY MAXI 24 CODE
    monthly_outbursts2=[]
    if len(monthly_detection2)!=0:
        ob=[monthly_detection_time2[0]]
        for i in range(len(monthly_detection_time2)):
            counter=0
            j=i
            while j<len(monthly_detection_time2) and monthly_detection_time2[j]<(monthly_detection_time2[i]+63): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(monthly_detection_time2[i])
                if ob[1]!=ob[0]:
                    monthly_outbursts2.append(ob)
                if i<len(monthly_detection_time2)-1:
                    ob=[monthly_detection_time2[i+1]]
                    
                    
                    
    #LOAD MAXI 410
    daily_counts3 = np.loadtxt(toload2, usecols = (5,))  #This will only give the count rate data (column index 5)
    daily_error3 = np.loadtxt(toload2, usecols = (6,))  #This will only give the count rate error data (column index 6)
    time3 = np.loadtxt(toload2, usecols = (0,))
    daily_mean3 = biweight_location(daily_counts3)
    daily_std3 = biweight_scale(daily_counts3)
    filtered_daily_data3 = sigma_clip(daily_counts3, sigma = 3)
    daily_detection3 = []
    daily_non_detection3 = []
    daily_detection_time3 = []
    for i in range(len(daily_counts3)):
        if daily_counts3[i] >= daily_mean3 + (3*daily_error3[i]):
            daily_detection3.append(daily_counts3[i])
            daily_detection_time3.append(time3[i])
        elif daily_counts3[i] < daily_mean3 + (3*daily_error3[i]):
            daily_non_detection3.append(daily_counts3[i])
                    
    
    #3a DAILY MAXI
    daily_outbursts3=[]
    if len(daily_detection3)!=0:
        ob=[daily_detection_time3[0]]
        for i in range(len(daily_detection_time3)):
            counter=0
            j=i
            while j<len(daily_detection_time3) and daily_detection_time3[j]<(daily_detection_time3[i]+7): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(daily_detection_time3[i])
                if ob[1]!=ob[0]:
                    daily_outbursts3.append(ob)
                if i<len(daily_detection_time3)-1:
                    ob=[daily_detection_time3[i+1]]

              
    #FOR GETTING THE WEEKLY AVERAGED COUNT DATA                      
    data_week_count = []
    data_week_error = []
    u = time3[0]
    i = 0
    wkly_counts3 = []
    wkly_error3 = []
    wkly_times3 = []  
    while i<=(len(time3)-1):
        if time3[i]-u<7:
            data_week_count.append(daily_counts3[i])
            data_week_error.append(daily_error3[i])
            i+=1
        else:
            if len(data_week_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
                sum_toAdd = np.sum(1/(np.square(data_week_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            wkly_counts3.append(toAdd)
            wkly_error3.append(toAdd_err)
            wkly_times3.append(u+3)
            u+=7
            data_week_count = []
            data_week_error = []    
    if len(data_week_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
        sum_toAdd = np.sum(1/(np.square(data_week_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    wkly_counts3.append(toAdd)
    wkly_error3.append(toAdd_err)
    wkly_times3.append(u+3)    
    weekly_counts3 = ma.masked_invalid(np.array(wkly_counts3))
    weekly_error3 = ma.masked_invalid(np.array(wkly_error3))
    weekly_times3 = np.array(wkly_times3)
                    
                    
    #GETTING THE COUNT RATE DATA DIVISION FOR WEEKLY-AVERAGED DATA
    weekly_mean3 = biweight_location(weekly_counts3)
    weekly_std3 = biweight_scale(weekly_counts3)
    filtered_weekly_data3 = sigma_clip(weekly_counts3, sigma = 3)
    weekly_detection3 = []
    weekly_non_detection3 = []
    weekly_detection_time3 = []
    for i in range(len(weekly_counts3)):
        if weekly_counts3[i] >= weekly_mean3 + (3*weekly_error3[i]):
            weekly_detection3.append(weekly_counts3[i])
            weekly_detection_time3.append(weekly_times3[i])
        elif weekly_counts3[i] < weekly_mean3 + (3*weekly_error3[i]):
            weekly_non_detection3.append(weekly_counts3[i])
                    
                    
    #3a WEEKLY MAXI 410 
    weekly_outbursts3=[]
    if len(weekly_detection3)!=0:
        ob=[weekly_detection_time3[0]]
        for i in range(len(weekly_detection_time3)):
            counter=0
            j=i
            while j<len(weekly_detection_time3) and weekly_detection_time3[j]<(weekly_detection_time3[i]+21): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(weekly_detection_time3[i])
                if ob[1]!=ob[0]:
                    weekly_outbursts3.append(ob)
                if i<len(weekly_detection_time3)-1:
                    ob=[weekly_detection_time3[i+1]]
                    

    #FOR GETTING THE MONTHLY AVERAGED COUNT DATA                   
    data_month_count = [] 
    data_month_error = []
    u=time3[0]
    i=0
    mth_counts3=[]
    mth_error3 = []
    mth_times3 = []       
    while i<=len(time3)-1:
        if time3[i]-u<21:
            data_month_count.append(daily_counts3[i])
            data_month_error.append(daily_error3[i])
            i+=1
        else:
            if len(data_month_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
                sum_toAdd = np.sum(1/(np.square(data_month_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            mth_counts3.append(toAdd)
            mth_error3.append(toAdd_err)
            mth_times3.append(u+10)
            u+=21
            data_month_count = []
            data_month_error = []
    if len(data_month_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
        sum_toAdd = np.sum(1/(np.square(data_month_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    mth_counts3.append(toAdd)
    mth_error3.append(toAdd_err)
    mth_times3.append(u+10)
    monthly_counts3= ma.masked_invalid(np.array(mth_counts3))
    monthly_error3 = ma.masked_invalid(np.array(mth_error3))
    monthly_times3 = np.array(mth_times3)
                    
                    
    #GETTING THE COUNT RATE DATA DIVISION FOR MONTHLY-AVERAGED DATA
    monthly_mean3 = biweight_location(monthly_counts3)
    monthly_std3 = biweight_scale(monthly_counts3)
    filtered_monthly_data3 = sigma_clip(monthly_counts3, sigma = 3)
    monthly_detection3 = []
    monthly_non_detection3 = []
    monthly_detection_time3 = []
    for i in range(len(monthly_counts3)):
        if monthly_counts3[i] >= monthly_mean3 + (3*monthly_error3[i]):
            monthly_detection3.append(monthly_counts3[i])
            monthly_detection_time3.append(monthly_times3[i])
        elif monthly_counts3[i] < monthly_mean3 + (3*monthly_error3[i]):
            monthly_non_detection3.append(monthly_counts3[i])
                    
                    
    #3a MONTHLY MAXI 410
    monthly_outbursts3=[]
    if len(monthly_detection3)!=0:
        ob=[monthly_detection_time3[0]]
        for i in range(len(monthly_detection_time3)):
            counter=0
            j=i
            while j<len(monthly_detection_time3) and monthly_detection_time3[j]<(monthly_detection_time3[i]+63): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(monthly_detection_time3[i])
                if ob[1]!=ob[0]:
                    monthly_outbursts3.append(ob)
                if i<len(monthly_detection_time3)-1:
                    ob=[monthly_detection_time3[i+1]]
                    
                    
                    
    #LOAD MAXI 210
    daily_counts4 = daily_counts3+daily_counts2
    daily_error4 = daily_error3+daily_error2
    time4 = time3
    daily_mean4 = biweight_location(daily_counts4)
    daily_std4 = biweight_scale(daily_counts4)
    filtered_daily_data4 = sigma_clip(daily_counts4, sigma = 3)
    daily_detection4 = []
    daily_non_detection4 = []
    daily_detection_time4 = []
    for i in range(len(daily_counts4)):
        if daily_counts4[i] >= daily_mean4 + (3*daily_error4[i]):
            daily_detection4.append(daily_counts4[i])
            daily_detection_time4.append(time4[i])
        elif daily_counts4[i] < daily_mean4 + (3*daily_error4[i]):
            daily_non_detection4.append(daily_counts4[i])
                    
                    
    #3a DAILY MAXI
    daily_outbursts4=[]
    if len(daily_detection4)!=0:
        ob=[daily_detection_time4[0]]
        for i in range(len(daily_detection_time4)):
            counter=0
            j=i
            while j<len(daily_detection_time4) and daily_detection_time4[j]<(daily_detection_time4[i]+7): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(daily_detection_time4[i])
                if ob[1]!=ob[0]:
                    daily_outbursts4.append(ob)
                if i<len(daily_detection_time4)-1:
                    ob=[daily_detection_time4[i+1]]
                    
                    
    #FOR GETTING THE WEEKLY AVERAGED COUNT DATA   
    data_week_count = []
    data_week_error = []
    u = time4[0]
    i = 0
    wkly_counts4 = []
    wkly_error4 = []
    wkly_times4 = []    
    while i<=(len(time4)-1):
        if time4[i]-u<7:
            data_week_count.append(daily_counts4[i])
            data_week_error.append(daily_error4[i])
            i+=1
        else:
            if len(data_week_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
                sum_toAdd = np.sum(1/(np.square(data_week_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            wkly_counts4.append(toAdd)
            wkly_error4.append(toAdd_err)
            wkly_times4.append(u+3)
            u+=7
            data_week_count = []
            data_week_error = []
    if len(data_week_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
        sum_toAdd = np.sum(1/(np.square(data_week_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    wkly_counts4.append(toAdd)
    wkly_error4.append(toAdd_err)
    wkly_times4.append(u+3)
    weekly_counts4 = ma.masked_invalid(np.array(wkly_counts4))
    weekly_error4 = ma.masked_invalid(np.array(wkly_error4))
    weekly_times4 = np.array(wkly_times4)
                    
                    
    #GETTING THE COUNT RATE DATA DIVISION FOR WEEKLY-AVERAGED DATA
    weekly_mean4 = biweight_location(weekly_counts4)
    weekly_std4 = biweight_scale(weekly_counts4)
    filtered_weekly_data4 = sigma_clip(weekly_counts4, sigma = 3)
    weekly_detection4 = []
    weekly_non_detection4 = []
    weekly_detection_time4 = []
    for i in range(len(weekly_counts4)):
        if weekly_counts4[i] >= weekly_mean4 + (3*weekly_error4[i]):
            weekly_detection4.append(weekly_counts4[i])
            weekly_detection_time4.append(weekly_times4[i])
        elif weekly_counts4[i] < weekly_mean4 + (3*weekly_error4[i]):
            weekly_non_detection4.append(weekly_counts4[i])
                    
                    
    #3a WEEKLY MAXI 210
    weekly_outbursts4=[]
    if len(weekly_detection4)!=0:
        ob=[weekly_detection_time4[0]]
        for i in range(len(weekly_detection_time4)):
            counter=0
            j=i
            while j<len(weekly_detection_time4) and weekly_detection_time4[j]<(weekly_detection_time4[i]+21): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(weekly_detection_time4[i])
                if ob[1]!=ob[0]:
                    weekly_outbursts4.append(ob)
                if i<len(weekly_detection_time4)-1:
                    ob=[weekly_detection_time4[i+1]]
                    
                    
    #FOR GETTING THE MONTHLY AVERAGED COUNT DATA   
    data_month_count = []
    data_month_error = []
    u=time4[0]
    i=0
    mth_counts4=[]
    mth_error4 = []
    mth_times4 = []     
    while i<=len(time4)-1:
        if time4[i]-u<21:
            data_month_count.append(daily_counts4[i])
            data_month_error.append(daily_error4[i])
            i+=1
        else:
            if len(data_month_count)==0:
                toAdd=0
            else:
                toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
                sum_toAdd = np.sum(1/(np.square(data_month_error)))
                toAdd_err = np.sqrt(1./sum_toAdd)
            mth_counts4.append(toAdd)
            mth_error4.append(toAdd_err)
            mth_times4.append(u+10)
            u+=21
            data_month_count = []
            data_month_error = []
    if len(data_month_count)==0:
        toAdd=0
    else:
        toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
        sum_toAdd = np.sum(1/(np.square(data_month_error)))
        toAdd_err = np.sqrt(1./sum_toAdd)
    mth_counts4.append(toAdd)
    mth_error4.append(toAdd_err)
    mth_times4.append(u+10)
    monthly_counts4= ma.masked_invalid(np.array(mth_counts4))
    monthly_error4 = ma.masked_invalid(np.array(mth_error4))
    monthly_times4 = np.array(mth_times4)
                    
                    
    #GETTING THE COUNT RATE DATA DIVISION FOR MONTHLY-AVERAGED DATA
    monthly_mean4 = biweight_location(monthly_counts4)
    monthly_std4 = biweight_scale(monthly_counts4)
    filtered_monthly_data4 = sigma_clip(monthly_counts4, sigma = 3)
    monthly_detection4 = []
    monthly_non_detection4 = []
    monthly_detection_time4 = []
    for i in range(len(monthly_counts4)):
        if monthly_counts4[i] >= monthly_mean4 + (3*monthly_error4[i]):
            monthly_detection4.append(monthly_counts4[i])
            monthly_detection_time4.append(monthly_times4[i])
        elif monthly_counts4[i] < monthly_mean4 + (3*monthly_error4[i]):
            monthly_non_detection4.append(monthly_counts4[i])
                    
                    
    #3a MONTHLY MAXI 210
    monthly_outbursts4=[]
    if len(monthly_detection4)!=0:
        ob=[monthly_detection_time4[0]]
        for i in range(len(monthly_detection_time4)):
            counter=0
            j=i
            while j<len(monthly_detection_time4) and monthly_detection_time4[j]<(monthly_detection_time4[i]+63): #What's after the plus is Y
                counter+=1
                j+=1
            if counter<2: #What's after the < is X
                ob.append(monthly_detection_time4[i])
                if ob[1]!=ob[0]:
                    monthly_outbursts4.append(ob)
                if i<len(monthly_detection_time4)-1:
                    ob=[monthly_detection_time4[i+1]]
                    

    #TOTAL MAXI DAILY OUTBURST LIST
    MAXI_daily_combo = daily_outbursts2 + daily_outbursts3 + daily_outbursts4
    total_MAXI_daily = ob1.merge(MAXI_daily_combo)

    #TOTAL MAXI WEEKLY OUTBURST LIST
    MAXI_weekly_combo = weekly_outbursts2 + weekly_outbursts3 + weekly_outbursts4
    total_MAXI_weekly = ob1.merge(MAXI_weekly_combo)

    #TOTAL MAXI MONTHLY OUTBURST LIST
    MAXI_monthly_combo = monthly_outbursts2 + monthly_outbursts3 + monthly_outbursts4
    total_MAXI_monthly = ob1.merge(MAXI_monthly_combo) 

    #TOTAL MAXI 2-4keV OUTBURST LIST
    MAXI_24_combo = daily_outbursts2 + weekly_outbursts2 + monthly_outbursts2
    total_MAXI_24 = ob1.merge(MAXI_24_combo)    

    #TOTAL MAXI 4-10keV OUTBURST LIST
    MAXI_410_combo = daily_outbursts3 + weekly_outbursts3 + monthly_outbursts3
    total_MAXI_410 = ob1.merge(MAXI_410_combo)    

    #TOTAL MAXI 2-10keV OUTBURST LIST
    MAXI_210_combo = daily_outbursts4 + weekly_outbursts4 + monthly_outbursts4
    total_MAXI_210 = ob1.merge(MAXI_210_combo) 

    #FINAL MAXI OUTBURST LIST
    energy_combo = total_MAXI_24 + total_MAXI_410 + total_MAXI_210
    final_MAXI = ob1.merge(energy_combo)    

    #TRUE FINAL OUTBURST LIST OF BOTH SWIFT AND MAXI
    data_combo = final_Swift + final_MAXI
    final_outbursts = ob1.merge(data_combo)
                          
    with open('outbursts_per_sources.txt', 'a') as f:
        f.write(str(hohoho))
        f.write(":")
        f.write('\n')
        f.write("Outbursts from all MAXI daily data:")
        f.write(str(total_MAXI_daily))
        f.write('\n')
        f.write("Outbursts from all MAXI weekly data:")
        f.write(str(total_MAXI_weekly))
        f.write('\n')
        f.write("Outbursts from all MAXI monthly data:")
        f.write(str(total_MAXI_monthly))
        f.write('\n')
        f.write("Outbursts from MAXI 2 - 4 KeV data:")
        f.write(str(total_MAXI_24))
        f.write('\n')
        f.write("Outbursts from MAXI 4 - 10 KeV data:")
        f.write(str(total_MAXI_410))
        f.write('\n')
        f.write("Outbursts from MAXI 2 - 10 KeV data:")
        f.write(str(total_MAXI_210))
        f.write('\n')
        f.write("Outbursts from all data:")
        f.write(str(final_MAXI))
        f.write('\n')
        f.write('\n')
        f.write('\n')
        f.write('\n')

C:\Users\ngutm\AppData\Local\Temp\ipykernel_20012\966373184.py:66: RuntimeWarning: divide by zero encountered in true_divide
  toAdd=np.average(data_week_count, weights = 1/(np.square(data_week_error)))
C:\Users\ngutm\anaconda3\lib\site-packages\numpy\lib\function_base.py:412: RuntimeWarning: invalid value encountered in multiply
  avg = np.multiply(a, wgt, dtype=result_dtype).sum(axis)/scl
C:\Users\ngutm\AppData\Local\Temp\ipykernel_20012\966373184.py:67: RuntimeWarning: divide by zero encountered in true_divide
  sum_toAdd = np.sum(1/(np.square(data_week_error)))
C:\Users\ngutm\AppData\Local\Temp\ipykernel_20012\966373184.py:137: RuntimeWarning: divide by zero encountered in true_divide
  toAdd=np.average(data_month_count, weights = 1/(np.square(data_month_error)))
C:\Users\ngutm\anaconda3\lib\site-packages\numpy\lib\function_base.py:412: RuntimeWarning: invalid value encountered in multiply
  avg = np.multiply(a, wgt, dtype=result_dtype).sum(axis)/scl
C:\Users\ngutm\AppData\Local\Te